In [12]:
%pip install avro
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json
import time
import requests

Note: you may need to restart the kernel to use updated packages.


In [13]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [14]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [15]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [16]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = '35.240.149.229:9092'

In [17]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [18]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [19]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [20]:
class TransactionVerifier:
    def __init__(self):
        self.token = "42cdee0e79c04bd1ffd5f4c4efe59a89"
        self.vid = "V411709"	
        self.submit_schema = submitschema
        self.result_schema = resultschema
        self.tx_schema = txschema
        self.gg_formurl = ""

    def read_transaction(self, transaction):
        # Extract the transaction information
        txid = transaction['txid']
        payer = transaction['payer']
        payee = transaction['payee']
        amount = transaction['amount']

        # Generate the signature using gen_signature function 
        signature = gen_signature(txid, payer, payee, amount, self.token)

        submit_data = {'vid': self.vid , 'txid': txid, 'signature': signature}
        
        data = serialize(self.submit_schema, submit_data)
        print(submit_data)
        print(data)
        producer.send('submit', data)
    
    def receive_confirmation(self, result):
        # Check if the result code is 200 and the vid and txid match
        if result['code'] == 200 and result['vid'] == self.vid and result['txid'] == txid:
            # Send the confirmation to Google form
            requests.post(self.gg_formurl, data={'entry.your_field_id': 'Confirmation'})


In [21]:
trxVerifier = TransactionVerifier()

In [22]:
print('Running Rst Consumer... with AVRO')
for message in resultconsumer:
    timestamp,vid,txid,checksum,code,message = message.value.values()
    print(timestamp,vid,txid,checksum,code,message)
    

Running Rst Consumer... with AVRO
1679676549 V411709 TX09207 740fffcd6beaf306c8e35a0d20bebadc 200 Confirm
1679676556 V411709 TX04679 7655a81624a1b79187c7006ca63d5d45 200 Confirm
1679676561 V411709 TX01319 6056aea28b9101aed97933135c15439c 200 Confirm
1679676567 V411709 TX08901 3d5a919b66a79a782e3a0acf768f1413 200 Confirm
1679676576 V411709 TX04551 32652356563350269fe12847898e5125 200 Confirm
1679676585 V411709 TX09356 4682c629953138fa784a6aa237e0dad7 200 Confirm
1679676593 V411709 TX02257 6996aab5c6d5683cfa0557442b27ad85 200 Confirm
1679676600 V411709 TX08317 b184a1d9008ea2ce5625e1bfb74a5c18 200 Confirm
1679676607 V411709 TX02615 18a6af765ecb325bbd55a617976ca634 200 Confirm
1679676615 V411709 TX05129 3d3807e2a65bf635eb6589342dbb3a2a 200 Confirm
1679676624 V411709 TX04455 259c280968772b45710d9a9932a32c3f 200 Confirm
1679676631 V411709 TX01962 70128af203257163ceaa78c34eaf963f 200 Confirm
1679676637 V411709 TX01730 7533afaefbc4788f5a6300babd8f75ee 200 Confirm
1679676646 V411709 TX06982 15f

KeyboardInterrupt: 